In [9]:
# Latent Dirichlet Allocation (LDA) with Python for Elliston Archive
import matplotlib
%pylab inline
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
sp_stop = stopwords.words('spanish')
from nltk import tokenize
from nltk.stem.porter import PorterStemmer
from gensim import corpora
from gensim import models, corpora
import glob

Populating the interactive namespace from numpy and matplotlib


In [9]:
# collect documents and perform simple cleaning to imporve topic model generation

p_stemmer = PorterStemmer() #use stemming to reduce topically similar words to their root

lst_filenames = []
for filename in glob.glob('data/*.txt'):
    lst_filenames.append(filename)

In [35]:
import io
def doc_preprocessor(lst_filenames):
    files = []
    for filename in lst_filenames:
        f = io.open(filename, 'r+', encoding='utf-8')
        read_data = f.read() #read in raw data
        file = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',filename) #remove URLs
        file = re.sub(r'[^\x00-\x7F]+','', file)
        file = re.sub(r'[^\w\s]','', file) #remove elements that are not words
        file = file.lower() #turn all words lowercase
        file = word_tokenize(file) #tokenize the data into its atomic elements 
        file = [i for i in file if not i in en_stop] #remove stopwords
        file = [p_stemmer.stem(i) for i in file] #stem tokens
        files.append(file)
        f.close()
    return files

In [36]:
docs = doc_preprocessor(lst_filenames)

In [39]:
dictionary = corpora.Dictionary(docs) # turn tokenized documents into a id <-> term dictionary
dictionary.compactify() #Assign new word ids to all words, shrinking gaps. Eliminate potential imaninary number representations
# dictionary.save('./warfarinldamodel_dict.dict')  # store the dictionary, for future reference
corpus = [dictionary.doc2bow(text) for text in docs] # convert tokenized documents into a document-term matrix

In [40]:
# Set training parameters.
num_topics = 35
chunksize = 500 # size of the doc looked at every pass
passes = 100 # number of passes through documents
iterations = 100
eval_every = 1  # Don't evaluate model perplexity, takes too much time.


In [41]:
##### Hyperparamter options and default values for gensim LDA model
# gensim.models.ldamodel.LdaModel(corpus=None, num_topics=100, id2word=None, distributed=False, chunksize=2000, \
# passes=1, update_every=1, alpha='symmetric', eta=None, decay=0.5, offset=1.0, eval_every=10, iterations=50, \
# gamma_threshold=0.001, minimum_probability=0.01, random_state=None, ns_conf=None, minimum_phi_value=0.01, \
# per_word_topics=False, callbacks=None, dtype=<type 'numpy.float32'>)

%time ldamodel = models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word = dictionary, chunksize=chunksize, passes=passes, eval_every = eval_every, iterations = iterations)

Wall time: 3d 1h 54min 17s


In [8]:
# keyword = ''
# print(ldamodel.get_term_topics('keyword'))
# print(ldamodel.get_topic_terms(ldamodel.get_term_topics('keyword')[0][0], topn=30))

[(10, 0.014984129), (11, 0.044843286), (12, 0.024069669), (15, 0.24911822), (20, 0.068382934), (23, 0.035111353)]
[(323, 0.057552308), (97, 0.053632706), (1042, 0.031103043), (3179, 0.029733561), (3178, 0.029733561), (3180, 0.029733561), (3176, 0.029733561), (3177, 0.029733561), (330, 0.023668991), (492, 0.022669429), (979, 0.021218568), (207, 0.019862682), (577, 0.016152158), (117, 0.01533929), (989, 0.012882323), (901, 0.0115640825), (41, 0.011399057), (1591, 0.011334097), (925, 0.011334097), (932, 0.010194605), (126, 0.010135128), (3100, 0.00996462), (820, 0.00996462), (3147, 0.00996462), (3164, 0.00996462), (3163, 0.00996462), (3162, 0.00996462), (3159, 0.00996462), (3160, 0.00996462), (3101, 0.00996462)]


In [42]:
# show the topics
topics = ldamodel.show_topics()
for topic in topics:
    print(topic)

(52, '0.289*"got" + 0.157*"that" + 0.071*"ur" + 0.040*"id" + 0.035*"boy" + 0.035*"stop" + 0.031*"left" + 0.019*"ago" + 0.016*"awww" + 0.016*"cuz"')
(38, '0.069*"hahaha" + 0.056*"forward" + 0.055*"kid" + 0.038*"almost" + 0.036*"woke" + 0.034*"breakfast" + 0.022*"quot" + 0.021*"chill" + 0.020*"save" + 0.015*"note"')
(43, '0.151*"ye" + 0.105*"read" + 0.079*"world" + 0.059*"book" + 0.048*"end" + 0.044*"true" + 0.040*"doesnt" + 0.019*"sent" + 0.016*"mood" + 0.015*"appl"')
(42, '0.119*"song" + 0.067*"two" + 0.053*"hot" + 0.040*"hang" + 0.039*"minut" + 0.033*"support" + 0.028*"tea" + 0.021*"addict" + 0.018*"awak" + 0.015*"sleepi"')
(36, '0.221*"watch" + 0.157*"home" + 0.053*"exam" + 0.033*"go" + 0.032*"im" + 0.025*"sing" + 0.025*"face" + 0.020*"suck" + 0.019*"award" + 0.019*"cook"')
(25, '0.065*"okay" + 0.062*"noth" + 0.052*"mom" + 0.047*"anyth" + 0.043*"worri" + 0.028*"lil" + 0.017*"past" + 0.016*"deserv" + 0.014*"color" + 0.013*"feet"')
(3, '0.220*"twitter" + 0.135*"right" + 0.089*"done" + 

In [43]:
#Visualize the LDA topics using pyLDAVis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [44]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

c:\python365\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
55     0.015962  0.122576       1        1  2.745447
51     0.009603  0.110796       2        1  2.597309
30     0.103123  0.046823       3        1  2.584507
56     0.232264  0.015888       4        1  2.372346
48     0.130747 -0.197603       5        1  2.182021
19    -0.092801 -0.032443       6        1  2.151231
37     0.107657  0.047224       7        1  2.083178
47    -0.092996 -0.030853       8        1  2.007994
49     0.012342  0.115703       9        1  1.956622
13    -0.092082 -0.031702      10        1  1.949638
52     0.011984 -0.062851      11        1  1.945921
50    -0.044487  0.037744      12        1  1.941770
5     -0.091977 -0.034149      13        1  1.899784
21     0.119658  0.006165      14        1  1.852512
16    -0.091928 -0.033158      15        1  1.851604
36     0.152372 -0.007369      16        1  1.851309
18    -0.092887 -0.032863      17        1  1.817759
35    -0.092506 -0.031284      18        1  1.807563
41     0.181453  0.020090      19        1  1.796553
59     0.042982 -0.154063      20        1  1.784712
44    -0.091449 -0.034524      21        1  1.762123
6     -0.064096  0.012147      22        1  1.732082
40    -0.055372  0.024984      23        1  1.723457
15     0.075293 -0.030293      24        1  1.708499
28     0.032103  0.133233      25        1  1.706102
11     0.022613  0.123355      26        1  1.618554
45     0.164362  0.058992      27        1  1.607436
54    -0.026897  0.064462      28        1  1.605000
1     -0.031429  0.057260      29        1  1.581827
57    -0.091773 -0.031257      30        1  1.571357
29     0.127340 -0.051657      31        1  1.568651
9     -0.092452 -0.032508      32        1  1.565566
58     0.007727 -0.123029      33        1  1.561546
14     0.027534 -0.139193      34        1  1.554707
33     0.066488 -0.089422      35        1  1.554239
32     0.063949  0.002812      36        1  1.512576
3     -0.092507 -0.030886      37        1  1.487027
12     0.007924  0.106300      38        1  1.480556
39     0.041765 -0.046215      39        1  1.456031
0     -0.018119  0.074028      40        1  1.453785
31     0.097330 -0.078413      41        1  1.445394
27    -0.008504  0.086238      42        1  1.424765
17    -0.063614  0.012072      43        1  1.418660
43    -0.092441 -0.030955      44        1  1.415494
8      0.026966 -0.072703      45        1  1.402293
7     -0.011637  0.079970      46        1  1.395650
20     0.034967 -0.037514      47        1  1.375368
2     -0.091010 -0.032647      48        1  1.371315
22     0.048101 -0.088565      49        1  1.371304
53    -0.091089 -0.033188      50        1  1.362301
46     0.012719  0.102944      51        1  1.354532
38    -0.014649  0.077519      52        1  1.347099
4     -0.091391 -0.031140      53        1  1.336437
24    -0.041954  0.045098      54        1  1.321765
42    -0.017277  0.074128      55        1  1.317331
10    -0.001035  0.090948      56        1  1.302121
26    -0.090381 -0.032472      57        1  1.275782
25    -0.047819  0.032845      58        1  1.265684
23    -0.003406 -0.109544      59        1  1.261714
34    -0.055360  0.022119      60        1  1.244098, topic_info=      Category           Freq         Term          Total  loglift  logprob
term                                                                       
111    Default  120949.000000          get  120949.000000  30.0000  30.0000
186    Default  108245.000000         good  108245.000000  29.0000  29.0000
101    Default   97201.000000         love   97201.000000  28.0000  28.0000
4      Default   95494.000000          day   95494.000000  27.0000  27.0000
51     Default   77016.000000        thank   77016.000000  26.0000  26.0000
168    Default   73033.000000         work   73033.000000  25.0000  25.0000
35     Default   76365.000000         like   76365.000000  24.0

In [45]:
p = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.save_html(p, keyword)

c:\python365\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
